In [1]:
import requests
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

INTERIM_PATH="data/interim/ENIGH"

In [3]:
os.chdir('..')

In [4]:
df_2022=pd.read_csv(f"{INTERIM_PATH}/2022.csv")
df_2020=pd.read_csv(f"{INTERIM_PATH}/2020.csv")
df_2018=pd.read_csv(f"{INTERIM_PATH}/2018.csv")
df_2016=pd.read_csv(f"{INTERIM_PATH}/2016.csv")

/var/folders/m5/26l5nfs54r94_664jm4k6zvr0000gn/T/ipykernel_14017/2970915959.py:1: DtypeWarning: Columns (11,12,13,14,15,16,17,18,27,31,32,33,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022=pd.read_csv(f"{INTERIM_PATH}/2022.csv")
/var/folders/m5/26l5nfs54r94_664jm4k6zvr0000gn/T/ipykernel_14017/2970915959.py:2: DtypeWarning: Columns (11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020=pd.read_csv(f"{INTERIM_PATH}/2020.csv")


# Graphs

In [12]:
def in_sonora(df):
    in_sonora= (df['folioviv']>=2600000000)&(df['folioviv']<2700000000)
    return in_sonora

def is_illiterate(df):
    if isinstance(df['alfabetism'], (int, float, complex)):
        is_illiterate=df['alfabetism']==1
    else:
        is_illiterate=df['alfabetism']=='1'
    return is_illiterate

# Grades:
def last_grade(df,grade):

    last_grade_dictionary={
    'preschool':1,
    'informal nursery school':2,
    'public daycare':3,
    'other daycare':5,
    'first grade':6,
    'second grade':7,
    'technical career with second grade':8,
    'highschool':9,
    'technical career with highschool':10,
    'normal':11,
    'professional':12,
    'masters or phd':13
    }

    if isinstance(df['nivel'], (int, float, complex)):
        last_grade=df['nivel']==last_grade_dictionary[grade]
    else:
        last_grade=df['nivel']==f'{last_grade_dictionary[grade]}'
    return last_grade

def approved_grade(df,grade):
    approved_grade_dictionary={
    'none':0,
    'preschool':1,
    'first grade':2,
    'second grade':3,
    'highschool':4,
    'technical career':6,
    'professional':7,
    'masters':8,
    'phd':9
    }
        
    if isinstance(df['nivelaprob'], (int, float, complex)):
        approved_grade=df['nivelaprob']==approved_grade_dictionary[grade]
    else:
        approved_grade=df['nivelaprob']==f'{approved_grade_dictionary[grade]}'
    return approved_grade
    
def is_assisting_school(df):
    if isinstance(df['asis_esc'], (int, float, complex)):
        is_assisting_school=df['asis_esc']==1
    else:
        is_assisting_school=df['asis_esc']=='1'
    return is_assisting_school


In [15]:
print(f"Amount of data obtained in Sonora in the year of 2016: {len(df_2016[in_sonora(df_2016)])}")
print(f"Amount of data obtained in Sonora in the year of 2018: {len(df_2018[in_sonora(df_2018)])}")
print(f"Amount of data obtained in Sonora in the year of 2020: {len(df_2020[in_sonora(df_2020)])}")
print(f"Amount of data obtained in Sonora in the year of 2022: {len(df_2022[in_sonora(df_2022)])}")

Amount of data obtained in Sonora in the year of 2016: 184
Amount of data obtained in Sonora in the year of 2018: 79
Amount of data obtained in Sonora in the year of 2020: 155
Amount of data obtained in Sonora in the year of 2022: 12332
